In [1]:
#Se instala la herramienta RDKit
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 45.6 MB/s eta 0:00:00


In [2]:
#Se instala el paquete duckdb que permite manipular la BD que inicialmente es muy grande
!pip install duckdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 92.2 MB/s eta 0:00:00


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#Se hace instalación de paquete de kaggle
!pip install -q kaggle

In [6]:
# Se usa la extensión de google colab para subir el token descargado de Kaggle el cual tiene el nombre "kaggle.json"
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"lhcastrop","key":"56e17656944c982f8a2a8576a52759d1"}'}

In [7]:
#Se crea directorio
!mkdir ~/.kaggle

In [8]:
#Se copia archivo en carpeta de colab
!cp kaggle.json ~/.kaggle/

In [9]:
!chmod 600 ~/.kaggle/kaggle.json

In [10]:
#Se carga el dataset desde kaggle en formato comprimido
! kaggle competitions download -c 'leash-BELKA'

100% 4.16G/4.16G [00:30<00:00, 162MB/s]
100% 4.16G/4.16G [00:30<00:00, 146MB/s]


In [21]:
#Se descomprimen los archivos en el directorio creado
! unzip leash-BELKA.zip -d '/content/drive/MyDrive/Inteligencia Artificial y Modelos/fase-2/Datos_BELKA'

Archive:  leash-BELKA.zip
replace /content/drive/MyDrive/Inteligencia Artificial y Modelos/fase-2/Datos_BELKA/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


**Preparación de Datos**

Los paquetes de datos de entrenamiento y prueba son definidos en los archivos .parquet. Se usa la herramienta duckdb para escanear la busqueda a traves de conjuntos de entramiento grandes. Para comenzar solo se toman un numero igual de muestras positivas y negativas.

Esta busqueda selecciona un numero igual de muestras donde los enlaces son 0 (no-enlazado) y 1 (enlazado), limitado para 30000 cada uno para evitar el sesgo del modelo hacia una clase particular.

In [13]:
from google.colab import drive
drive.mount('/content/drive')
import duckdb
import pandas as pd

train_path = '/content/drive/MyDrive/Inteligencia Artificial y Modelos/fase-2/Datos_BELKA/train.parquet'
test_path = '/content/drive/MyDrive/Inteligencia Artificial y Modelos/fase-2/Datos_BELKA/test.parquet'


con = duckdb.connect()

df = con.query(f"""(SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 0
                        ORDER BY random()
                        LIMIT 30000)
                        UNION ALL
                        (SELECT *
                        FROM parquet_scan('{train_path}')
                        WHERE binds = 1
                        ORDER BY random()
                        LIMIT 30000)""").df()

con.close()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [14]:
df.head()

,id,buildingblock1_smiles,buildingblock2_smiles,buildingblock3_smiles,molecule_smiles,protein_name,binds
0,292616505,O=C(O)[C@H]1C[C@H](O)CN1C(=O)OCC1c2ccccc2-c2cc...,Cl.NC[C@@H]1C[C@@H]2O[C@H]1[C@H]1C[C@H]12,CN(C)C(CN)c1ccco1,CN(C)C(CNc1nc(NC[C@@H]2C[C@@H]3O[C@H]2[C@H]2C[...,BRD4,0
1,8494584,C=CCC[C@@H](NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,Nc1cc(F)c(F)cc1[N+](=O)[O-],NCC1(c2ccc3c(c2)OCO3)CCCCC1,C=CCC[C@@H](Nc1nc(NCC2(c3ccc4c(c3)OCO4)CCCCC2)...,BRD4,0
2,18324470,CC(C)CC(NC(=O)OCC1c2ccccc2-c2ccccc21)C(=O)O,Nc1ccc(Cl)c(C(F)(F)F)c1,Nc1ccc(Br)c(F)n1,CC(C)CC(Nc1nc(Nc2ccc(Cl)c(C(F)(F)F)c2)nc(Nc2cc...,sEH,0
3,160452511,O=C(Nc1cc(Br)ccc1C(=O)O)OCC1c2ccccc2-c2ccccc21,COc1ncccc1N,CC1CN(c2cc(CN)ccn2)CCO1,COc1ncccc1Nc1nc(NCc2ccnc(N3CCOC(C)C3)c2)nc(Nc2...,HSA,0
4,291383726,O=C(O)[C@H]1COCCN1C(=O)OCC1c2ccccc2-c2ccccc21,Cl.Cl.NCc1cc2n(n1)CCCO2,Cc1nn(C)c2ncc(CN)cc12.Cl,Cc1nn(C)c2ncc(CNc3nc(NCc4cc5n(n4)CCCO5)nc(N4CC...,sEH,0


**Procesamiento de Caracteristicas**

Se toman los SMILEs correspondientes a cada molecula completamente ensamblada (molecule_smiles) y se generan los ECFPs para estas. Podriamos escoger diferentes radios o bits, pero 2 y 1024 son los mas estandar.

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import OneHotEncoder


# Se convierten los SMILEs en moleculas RDKit
df['molecule'] = df['molecule_smiles'].apply(Chem.MolFromSmiles)

#Se generan las ECFPs (Extended-Conectivity Fingerprints- Huellas Dactilares de Conectividad Extendida)
def generate_ecfp(molecule, radius=2, bits=1024):
    if molecule is None:
        return None
    return list(AllChem.GetMorganFingerprintAsBitVect(molecule, radius, nBits=bits))

df['ecfp'] = df['molecule'].apply(generate_ecfp)


**Modelo de Entrenamiento**

In [16]:
import joblib

# Se realiza la codificación One-hot a los nombres de las proteinas almacenadas en protein_name
onehot_encoder = OneHotEncoder(sparse_output=False)
protein_onehot = onehot_encoder.fit_transform(df['protein_name'].values.reshape(-1, 1))

# Se combinan las ECFPs y los nombres de la proteinas codificadas
X = [ecfp + protein for ecfp, protein in zip(df['ecfp'].tolist(), protein_onehot.tolist())]
y = df['binds'].tolist()

# Se dividen los datos en conjunto de prueba y entrenamiento
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Se crea y entrena el Modelo Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Se guarda el modelo
#joblib.dump(rf_model, 'modelo_entrenado.pkl')

# Se realizan las predicciones  con el conjunto de prueba
y_pred_proba = rf_model.predict_proba(X_test)[:, 1]  # Probabilidad de la clase positiva

# Se calcula la precisión media promedio
map_score = average_precision_score(y_test, y_pred_proba)
print(f"Mean Average Precision (mAP): {map_score:.2f}")

Mean Average Precision (mAP): 0.96


**Almaecenamiento del Modelo**

In [17]:
import pickle
with open("/content/drive/MyDrive/Inteligencia Artificial y Modelos/fase-2/model.pkl", "wb") as f:
    pickle.dump(rf_model, f)

**Prueba del modelo con los datos de prueba**

El modelo Random Forest entrenado es usado para predecir las probabilidad de afinidad. Estas predicciones se guardan en un archivo CSV.

In [18]:
X_test_df=pd.DataFrame(X_test)
X_test_df.to_csv("/content/drive/MyDrive/Inteligencia Artificial y Modelos/fase-2/test_data_input.csv", index=False)
# Cargar el archivo CSV
df = pd.read_csv('/content/drive/MyDrive/Inteligencia Artificial y Modelos/fase-2/test_data_input.csv')

# Imprimir el contenido
#print(df)

y_test_df=pd.DataFrame(y_test)
y_test_df.to_csv("/content/drive/MyDrive/Inteligencia Artificial y Modelos/fase-2/test_data_target.csv", index=False)

# Cargar el archivo CSV
df = pd.read_csv('/content/drive/MyDrive/Inteligencia Artificial y Modelos/fase-2/test_data_target.csv')

# Imprimir el contenido
#print(df)

#Se carga el modelo previamente creado
with open('/content/drive/MyDrive/Inteligencia Artificial y Modelos/fase-2/model.pkl', 'rb') as f:
    _m = pickle.load(f)

In [19]:
#Se realiza la prediccion
_m.predict(X_test)

array([0, 1, 1, ..., 1, 1, 1])

In [20]:
_m.score(X_test, y_test)

0.8905